# Spotify API

### Goals:

- interact w/ spotify api to download erraything in a user's songs 
    - hopefully that's not the same user as the client tokens used for the api; if that ends up being necessary that's ok
- find "what's the most 'unique' song in top 100 listened-to?"

## Spotify API setup

Could use spotipy, but since jumping back in going to try and roll own

Working from here:
https://stmorse.github.io/journal/spotify-api.html

#### Get `Client ID` and `Client Secret` from spotify api page
located here:
https://developer.spotify.com/dashboard/applications

Signed in w/ personal spotify login

Client ID 1c2e33a8bebd41fbbb8a1ecf0e8c4273

Client Secret 4964098dcc7b41f99c4178e6403645c1

### formatting POST requests

In [2]:
import requests

CLIENT_ID = '1c2e33a8bebd41fbbb8a1ecf0e8c4273'
CLIENT_SECRET = '4964098dcc7b41f99c4178e6403645c1'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST

auth_response = requests.post(AUTH_URL, {
    'grant_type' : 'client_credentials', 
    'client_id' : CLIENT_ID,
    'client_secret' : CLIENT_SECRET,   
})

# convert response to json
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

### Setting correct headers for POST request

In [3]:
headers = {
    'Authorization' : 'Bearer {token}'.format(token=access_token)
    
}

### test so far: grab audio features from a single song

In [3]:
# base URL of all spotify api endpoints

BASE_URL = 'https://api.spotify.com/v1/'

# TRACK ID from spotify URI obtained from regular spotify web page
track_id = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request w/ proper header

r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

r = r.json()

r

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [6]:
audio_analysis = requests.get(BASE_URL + 'audio-analysis/' + track_id, headers=headers).json()

In [7]:
audio_analysis.keys()

audio_analysis['track']

{'num_samples': 5179756,
 'duration': 234.90958,
 'sample_md5': '',
 'offset_seconds': 0,
 'window_seconds': 0,
 'analysis_sample_rate': 22050,
 'analysis_channels': 1,
 'end_of_fade_in': 0.0,
 'start_of_fade_out': 230.31293,
 'loudness': -4.359,
 'tempo': 119.878,
 'tempo_confidence': 0.13,
 'time_signature': 4,
 'time_signature_confidence': 1.0,
 'key': 0,
 'key_confidence': 0.578,
 'mode': 1,
 'mode_confidence': 0.585,
 'codestring': 'eJxdm1mi3SgMRLfiJZgZ9r-xPqfwe0n6o9OBa2MGqVQlkfLW1Vtrb3_eZ589du_vfkbvtvrYY53n1P2s0eqsZz7lrf625uiz0DyjPqPUd9fZntrf91m9n30YsfLUs3t527toNl-nudapqzytj5mPvK2882mz7--b_a1P26v68G6j8YVeDn-cd8zJF_pmpLpne08ZnX5_XJN1MLtnVOa1G-to7z7PHPxaeXGud-1nZlJn7VZoP6s40dXHYBH9WWu5xN7mS5MtqXs_h006fbzl2avx4eJG9d4e58PTu5ZV9vucwVhn71PqZjEMMxm7nHeewdPlXfX1bb68bJeXre51v62yf7Q7H5ujjN2mO13O3D6_OpN5n1I9Fva2sjeF93nzdctneZn8U5pHc_Y72bJWaFdmwbujtPXadtT6MmsGWLQ3z_PHab0dvt9fj2DzsVMr4_fJ8pjsqKze9uH7bMbr6nl_VNbHKnvl1ckExsoEmWdln8pkYQ87t-d2qwtnpyGNNd4z-N7UsIa7wsR5fjlfTMd1sJ1lNVZFe61WN9_fuzgfhh

### Ok, we can get stuff from the API; let's work on getting stuff from a specific user

according to here, should use `/users/{user_id}/playlists` to grab a user's playlists:

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-list-users-playlists

In [4]:
# user_ID according to above link is just the public-facing regular account name

user_id = 'bothsidesdoit'

# actual GET request w/ proper header

r = requests.get(BASE_URL + 'users/' + user_id + '/playlists', headers=headers)

r = r.json()

### It returns something, let's explore it 

In [5]:
r.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
len(r['items'])

6

In [7]:
for item in r['items']:
    print(item['name'])

anytime
guess theme party
current
pump th' fuck up
good matt
krzyg


### eyyyy it works

are the songs in here that we can just grab?

In [19]:
r['items'][0]

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1oqsGOPQF8TqVtiFdfqkd6'},
 'href': 'https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6',
 'id': '1oqsGOPQF8TqVtiFdfqkd6',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b2734f0c0e8030e4e20ea5179765ab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b2734f0c0e8030e4e20ea5179765ab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b2734f0c0e8030e4e20ea5179765ab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 60}],
 'name': 'anytime',
 'owner': {'display_

### nope, but looks like under `tracks` it lists the URL to call to get the tracks, let's see

In [23]:
playlist_test_url = r['items'][0]['tracks']['href']

single_playlist_request = requests.get(playlist_test_url, headers=headers)

single_playlist_request = single_playlist_request.json()

single_playlist_request

{'href': 'https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6/tracks?offset=0&limit=100',
 'items': [{'added_at': '2020-06-23T21:31:28Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/bothsidesdoit'},
    'href': 'https://api.spotify.com/v1/users/bothsidesdoit',
    'id': 'bothsidesdoit',
    'type': 'user',
    'uri': 'spotify:user:bothsidesdoit'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/72PnPUc1qv9UjRPaGVZ1jq'},
       'href': 'https://api.spotify.com/v1/artists/72PnPUc1qv9UjRPaGVZ1jq',
       'id': '72PnPUc1qv9UjRPaGVZ1jq',
       'name': 'The Handsome Family',
       'type': 'artist',
       'uri': 'spotify:artist:72PnPUc1qv9UjRPaGVZ1jq'}],
     'available_markets': [],
     'external_urls': {'spotify': 'https://open.spotify.com/album/3u4rZDOy1c7N0otfHS0qTj'},
     'href': 'https://api.spotify.com/v1/albums/3u4rZ

### yuhp 
for each song in the playlist (the number in the brackets below), track url to call is located here:

In [28]:
single_playlist_request['items'][0]['track']['href']

'https://api.spotify.com/v1/tracks/4B4xOuj22g8WAMvTzPV7Bq'

### let's GET the data at the individual track url above

In [30]:
track_test_url = single_playlist_request['items'][0]['track']['href']

single_track_request = requests.get(track_test_url, headers=headers)

single_track_request = single_track_request.json()

single_track_request

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/72PnPUc1qv9UjRPaGVZ1jq'},
    'href': 'https://api.spotify.com/v1/artists/72PnPUc1qv9UjRPaGVZ1jq',
    'id': '72PnPUc1qv9UjRPaGVZ1jq',
    'name': 'The Handsome Family',
    'type': 'artist',
    'uri': 'spotify:artist:72PnPUc1qv9UjRPaGVZ1jq'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/3u4rZDOy1c7N0otfHS0qTj'},
  'href': 'https://api.spotify.com/v1/albums/3u4rZDOy1c7N0otfHS0qTj',
  'id': '3u4rZDOy1c7N0otfHS0qTj',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273b1e2efeb6be31bede74d415e',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02b1e2efeb6be31bede74d415e',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851b1e2efeb6be31bede74d415e',
    'width': 64}],
  'name': 'Singing Bones',
  'release_date': '2003-09-29',
  'r

### eyyy looks like we got a lot of track info here

doc for info in above GET request is here:

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track

In [39]:
len(single_playlist_request['items'])

26

### Could go a lot of ways here; let's put everything we've got together

for a given user:
- GET the playlists
  - save playlists in obj
- for each playlist:
  - go into playlist obj and for each track:
    - pull out desired information of each song
  - save song info in dataframe with playlist name

working backwards, first w/ the "for each playlist" part

In [51]:
# getting desired info for each track; change the int to change tracks
single_playlist_request['items'][0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [52]:
single_playlist_request['items'][0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/72PnPUc1qv9UjRPaGVZ1jq'},
    'href': 'https://api.spotify.com/v1/artists/72PnPUc1qv9UjRPaGVZ1jq',
    'id': '72PnPUc1qv9UjRPaGVZ1jq',
    'name': 'The Handsome Family',
    'type': 'artist',
    'uri': 'spotify:artist:72PnPUc1qv9UjRPaGVZ1jq'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/3u4rZDOy1c7N0otfHS0qTj'},
  'href': 'https://api.spotify.com/v1/albums/3u4rZDOy1c7N0otfHS0qTj',
  'id': '3u4rZDOy1c7N0otfHS0qTj',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273b1e2efeb6be31bede74d415e',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02b1e2efeb6be31bede74d415e',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851b1e2efeb6be31bede74d415e',
    'width': 64}],
  'name': 'Singing Bones',
  'release_date': '2003-09-29',
  'r

In [54]:
# list of keys from above dict to grab
track_info_dict = {
    'artist':[['artists'],[0],['name']],
    'album_name': [['album'],['name']],
    'album_release_date': [['album'],['release_date']],
    'album_total_tracks': [['album'],['total_tracks']],
    'duration_ms' : ['duration_ms'],
    'external_ids' : ['external_ids'],
    'track_name' : ['name'],
    'track_number' : ['track_number']
}

In [56]:
test = single_playlist_request['items'][0]['track']
test[track_info_dict['track_name'][0]]

'Far From Any Road'

In [64]:
pd.json_normalize(
    data=single_playlist_request['items'][0]['track'],
)

,artists,available_markets,disc_number,duration_ms,episode,explicit,href,id,is_local,name,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,[],1,166493,False,False,https://api.spotify.com/v1/tracks/4B4xOuj22g8W...,4B4xOuj22g8WAMvTzPV7Bq,False,Far From Any Road,...,3u4rZDOy1c7N0otfHS0qTj,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Singing Bones,2003-09-29,day,13,album,spotify:album:3u4rZDOy1c7N0otfHS0qTj,US2X80436005,https://open.spotify.com/track/4B4xOuj22g8WAMv...


In [71]:
import pandas as pd

pd.json_normalize(
    data=single_playlist_request['items'],
    record_path=['artists'],
    meta=['disc_number', ['height']]
)


KeyError: 'album.images'

In [84]:
r = requests.get(BASE_URL + 'users/' + user_id + '/playlists', headers=headers)

r = r.json()

[r['items'][n]['name'] for n in range(0,len(r['items']))]

['anytime',
 'guess theme party',
 'current',
 "pump th' fuck up",
 'good matt',
 'krzyg']

In [86]:
r['items'][2]['tracks']

{'href': 'https://api.spotify.com/v1/playlists/1Zd7dmudNrNwpfShtK6ywf/tracks',
 'total': 3}

In [87]:
playlist_test_url = r['items'][2]['tracks']['href']

single_playlist_request = requests.get(playlist_test_url, headers=headers)

single_playlist_request = single_playlist_request.json()

single_playlist_request

{'href': 'https://api.spotify.com/v1/playlists/1Zd7dmudNrNwpfShtK6ywf/tracks?offset=0&limit=100',
 'items': [{'added_at': '2017-12-15T19:33:29Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/bothsidesdoit'},
    'href': 'https://api.spotify.com/v1/users/bothsidesdoit',
    'id': 'bothsidesdoit',
    'type': 'user',
    'uri': 'spotify:user:bothsidesdoit'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7x83XhcMbOTl1UdYsPTuZM'},
       'href': 'https://api.spotify.com/v1/artists/7x83XhcMbOTl1UdYsPTuZM',
       'id': '7x83XhcMbOTl1UdYsPTuZM',
       'name': 'Tom Waits',
       'type': 'artist',
       'uri': 'spotify:artist:7x83XhcMbOTl1UdYsPTuZM'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'B

# Indidivual track analysis

In [4]:
track = requests.get(
    'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
    headers=headers
).json()

In [5]:
track.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])

In [6]:
track['track'].keys()

dict_keys(['num_samples', 'duration', 'sample_md5', 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence', 'codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version'])

In [24]:
track['beats'][0:5], track['bars'][0:5]

([{'start': 0.59909, 'duration': 0.5132, 'confidence': 0.752},
  {'start': 1.11229, 'duration': 0.52672, 'confidence': 0.394},
  {'start': 1.63902, 'duration': 0.5315, 'confidence': 0.249},
  {'start': 2.17051, 'duration': 0.51586, 'confidence': 0.125},
  {'start': 2.68637, 'duration': 0.51259, 'confidence': 0.189}],
 [{'start': 1.11229, 'duration': 2.08666, 'confidence': 0.646},
  {'start': 3.19895, 'duration': 2.04405, 'confidence': 0.537},
  {'start': 5.243, 'duration': 2.07919, 'confidence': 0.565},
  {'start': 7.32219, 'duration': 2.03038, 'confidence': 0.2},
  {'start': 9.35258, 'duration': 1.98788, 'confidence': 0.173}])